In [1]:
# Import neccesary programs
import requests
import bs4
from bs4 import BeautifulSoup
from lxml import html
import numpy as np

### Webscraping Functions

In [2]:
# Define functions for extracting information

def extract_location_from_result(loc):
    try:
        location = []
        for n in loc.findAll(class_='location'):
            location.append(n.text.strip())
        return location
    except:
        return np.nan   



In [3]:
def extract_company_from_result(comp):
    try:
        company = []
        for c in comp.findAll('span', class_='company'):
            company.append(c.text.strip())
        return company
    except:
        return np.nan

In [4]:
def extract_jobtitle_from_result(jobtitle):
    try:
        jobtitle = []
        for j in jobtitle.findAll(class_='jobtitle'):
            jobtitle.append(j.text.strip())
        return jobtitle
    except:
        return np.nan  

In [5]:
def extract_salary_from_result(sal):
    try: 
        salary = []
        for s in sal.findAll(class_='snip'):
            sal.append(s.nobr())
        return salary
    except:
        return np.nan


In [6]:
def extract_summary_from_results(desc):
    try:
        summary = []
        for m in desc.find('span', class_='summary'):
            desc.append(m.text.strip())
        return summary
    except:
        return np.nan

### Select Website 

In [8]:
# Run loop to get multiple results. Pass functions through results
Location = []
Job_Title = []
Company = []
Salary = []
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))

In [ ]:
city_list =[]
for city in set(cities):
    for i in range(city_count[city_count["city"]==city]["result_count"]):
        city_list.append(city)

In [ ]:
# Create df of all search results

df = pd.DataFrame(zip(city_list, location, company, title, salary, description))
df.columns = ["city", "location", "company", "title", "salary", "description"]
print df.shape
df.tail()

### Cleaning Salary Data